In [18]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '문서 내용 QA']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess

pass

#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess

            pass
            
            #### data preprocess end 

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = ''
            text += '##title:\n' + line['title'] + '\n' + '##passage:\n' + line['passage'] + '\n'
            for qa_pair in line['qa_pairs']:
                text += '##question:\n' + qa_pair['question'] + '\n'
                if qa_pair["answer"]["spans"] != None:
                    for span in qa_pair["answer"]["spans"]:
                        text += '##answer:\n' + span['text'] + '\n'
                elif qa_pair["answer"]["date"] != None:
                    text += '##date:\n'
                    if qa_pair["answer"]["date"]['year'] != None:
                        text += qa_pair["answer"]["date"]['year'] + '.'
                    if qa_pair["answer"]["date"]['month'] != None:
                        text += qa_pair["answer"]["date"]['month'] + '.'
                    if qa_pair["answer"]["date"]['day'] != None:
                        text += qa_pair["answer"]["date"]['day']
                    text += '\n'
                elif qa_pair["answer"]["number"] != None:
                    text += '##number:\n'
                    if qa_pair["answer"]['number']['calculation'] != None:
                        text += qa_pair["answer"]['number']['calculation'] + '='
                    if qa_pair["answer"]['number']['number'] != None:
                        text += qa_pair["answer"]['number']['number']
                    if qa_pair["answer"]['number']['unit'] != None:
                        text += qa_pair["answer"]['number']['unit']
                    text += '\n'
            data['text'] = text + '\n'
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end


            #### 문서 내용 QA
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['idx'] = line['idx']
            data['input']['category'] = line['category']
            data['input']['title'] = line['title']
            data['input']['passage'] = line['passage']
            for qa_pair in line['qa_pairs']:
                data['input']['query_id'] = qa_pair['query_id']
                data['input']['question'] = qa_pair['question']
                data['output']['answer'] = qa_pair['answer']

                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 내용 QA end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 2/2 [00:01<00:00,  1.10it/s]
